In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from metaflow import Flow, get_metadata
from matplotlib import pyplot as plt

In [31]:
run = Flow('TraderFlow').latest_successful_run
data = run.data.stocks
data

{'A': array([[47.68, 44.41, 41.01, ..., 44.1 , 43.17, 42.02],
        [44.41, 41.01, 38.47, ..., 43.17, 42.02, 42.4 ],
        [41.01, 38.47, 37.01, ..., 42.02, 42.4 , 42.25],
        ...,
        [88.62, 89.46, 90.12, ..., 82.56, 82.15, 83.52],
        [89.46, 90.12, 89.61, ..., 82.15, 83.52, 84.93],
        [90.12, 89.61, 90.13, ..., 83.52, 84.93, 84.82]]),
 'GOOG': array([[  49.98,   53.95,   54.5 , ...,   49.82,   50.6 ,   50.96],
        [  53.95,   54.5 ,   52.24, ...,   50.6 ,   50.96,   50.96],
        [  54.5 ,   52.24,   52.8 , ...,   50.96,   50.96,   52.47],
        ...,
        [1439.2 , 1451.7 , 1480.39, ..., 1434.23, 1485.94, 1447.07],
        [1451.7 , 1480.39, 1484.4 , ..., 1485.94, 1447.07, 1448.23],
        [1480.39, 1484.4 , 1485.95, ..., 1447.07, 1448.23, 1476.23]]),
 'INTC': array([[26.37, 27.88, 26.57, ..., 32.72, 32.06, 30.64],
        [27.88, 26.57, 26.8 , ..., 32.06, 30.64, 31.38],
        [26.57, 26.8 , 25.23, ..., 30.64, 31.38, 31.66],
        ...,
        [

In [30]:
len(data['MSFT'])

5044

In [22]:
(-0.38289362 * 235) + 132

42.01999930000001

In [23]:
from utils import normalize

In [28]:
normalize(np.array([np.array([42.02, 42.4 , 42.25, 83.52, 84.93, 84.82])]))

array([[-0.38289362, -0.3812766 , -0.38191489, -0.20629787, -0.20029787,
        -0.20076596]])

In [8]:
list(data.values())[0][:,-1]

array([42.02, 42.4 , 42.25, ..., 83.52, 84.93, 84.82])

In [18]:
all = np.concatenate([stocks[:,-1] for stocks in data.values()])
np.std(all)

235.50982183514304